In [40]:
from gym import Env
from gym.spaces import Discrete, Box
import numpy as np
import random
import matplotlib.pyplot as plt
import gym
import collections

In [41]:
dist=[0,1,1.848,2.414,2.613,2.414,1.848,1]
def timer(a1,a2):
    return dist[abs(a1-a2)]
r=[70,0,0,70,70,0,70,0]
rr=[8, 0, 0, 2, 2 , 0, 8, 0]


In [50]:
class forage_env(gym.Env):
    def __init__(self):
        self.action_space = Discrete(8)
        # Temperature array
        self.observation_space = Discrete(8)
        self.state=random.randint(0,7)
        # Total time per episode
        self.timer = 300
        self.rewards=[70,0,0,70,70,0,70,0]
        self.reward_rate=[8, 0, 0, 2, 2 , 0, 8, 0]
        self.E=[0,0,0,0,0,0,0,0]
        self.F=[]
        self.G=[]
        
    def step(self,action):
        self.F.append(action)
        a=self.state
        self.state = action
        self.timer-=1+timer(a,action)
        #print(self.timer)
        reward=self.rewards[action]
        for i in range(len(self.rewards)):
            if(i!=action):
                if(self.reward_rate[i]+self.rewards[i]>200):
                    self.rewards[i]=200
                else:   
                    self.rewards[i]=self.reward_rate[i]+self.rewards[i]
            else:
                self.rewards[i]=0.9*self.rewards[i]
                self.E[i]+=1
        
        if self.timer <= 0: 
            done = True
        else:
            done = False
        info = {}
        self.G.append(reward)
        #print(self.E)
        #print(reward)
        # Return step information
        return self.state, reward, done, info

    def render(self):
        # Implement
        pass
    
    def reset(self):
        self.state=random.randint(0,7)
        self.timer = 300
        self.rewards=[70,0,0,70,70,0,70,0]
        self.E=[0,0,0,0,0,0,0,0]
        self.F=[]
        self.G=[]
        
        return self.state

In [51]:
class Agent:
    def __init__(self):
        self.env = forage_env()
        self.state = self.env.reset()
        self.values = collections.defaultdict(float)
        #self.actioncount=[]

    def sample_env(self):
        action = self.env.action_space.sample()
        old_state = self.state
        new_state, reward, is_done, _ = self.env.step(action)
        self.state = self.env.reset() if is_done else new_state
        return old_state, action, reward, new_state

    def best_value_and_action(self, state):
        best_value, best_action = None, None
        for action in range(self.env.action_space.n):
            action_value = self.values[(state, action)]
            if best_value is None or best_value < action_value:
                best_value = action_value
                best_action = action
        return best_value, best_action

    def value_update(self, s, a, r, next_s):
        best_v, _ = self.best_value_and_action(next_s)
        new_v = r + GAMMA * best_v
        old_v = self.values[(s, a)]
        self.values[(s, a)] = old_v * (1-ALPHA) + new_v * ALPHA
        #print(self.values)
        #return self.values[(s, a)]

    def play_episode(self, env):
        total_reward = 0.0
        state = env.reset()
        while True:
            _, action = self.best_value_and_action(state)
            #self.actioncount.append(action)
            new_state, reward, is_done, _ = env.step(action)
            total_reward += reward
            if is_done:
                break
            state = new_state
        return total_reward 

In [56]:
step_counter=[]
step_=[]
reward_=[]
import gym
import collections
from tensorboardX import SummaryWriter
GAMMA = 1
ALPHA = 0.2
TEST_EPISODES = 1
if __name__ == "__main__":
    test_env = forage_env()
    agent = Agent()
    #writer = SummaryWriter(comment="-q-learning")

    iter_no = 0
    best_reward = 0.0
    while iter_no<10000:
        iter_no += 1
        s, a, r, next_s = agent.sample_env()
        agent.value_update(s, a, r, next_s)
        reward = 0.0
        for _ in range(TEST_EPISODES):
            reward += agent.play_episode(test_env)
            
        reward /= TEST_EPISODES
        #writer.add_scalar("reward", reward, iter_no)
        if reward > best_reward:
            print("Best reward updated %.3f -> %.3f" % (
                best_reward, reward))
            #print(agent.values)
            
            step_counter.append(test_env.E)
            step_.append(test_env.F)
            reward_.append(test_env.G)
            best_reward = reward
            
    #writer.close()

Best reward updated 0.000 -> 700.000
Best reward updated 700.000 -> 5980.007
Best reward updated 5980.007 -> 7789.067
Best reward updated 7789.067 -> 8360.451
Best reward updated 8360.451 -> 8509.825
Best reward updated 8509.825 -> 9890.361
Best reward updated 9890.361 -> 9989.216
Best reward updated 9989.216 -> 10046.927
Best reward updated 10046.927 -> 10402.234
Best reward updated 10402.234 -> 10443.333
Best reward updated 10443.333 -> 10489.144
Best reward updated 10489.144 -> 10501.479
Best reward updated 10501.479 -> 12964.899
Best reward updated 12964.899 -> 13110.348
Best reward updated 13110.348 -> 13164.899


In [57]:
step_counter            

[[299, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 75, 75],
 [0, 0, 25, 25, 25, 1, 26, 0],
 [53, 0, 0, 0, 0, 0, 53, 0],
 [52, 0, 0, 1, 0, 0, 53, 0],
 [31, 0, 0, 31, 0, 0, 31, 0],
 [31, 0, 0, 31, 0, 0, 32, 0],
 [32, 0, 0, 31, 0, 0, 31, 0],
 [33, 0, 0, 0, 32, 0, 32, 0],
 [33, 0, 0, 0, 33, 0, 32, 0],
 [32, 0, 0, 1, 32, 0, 32, 0],
 [33, 0, 0, 0, 32, 0, 33, 0],
 [27, 0, 0, 27, 27, 0, 27, 0],
 [27, 0, 0, 27, 28, 0, 27, 0],
 [27, 0, 0, 27, 27, 0, 28, 0]]

In [59]:
step_[8]
            

[0,
 4,
 6,
 0,
 4,
 6,
 0,
 4,
 6,
 0,
 4,
 6,
 0,
 4,
 6,
 0,
 4,
 6,
 0,
 4,
 6,
 0,
 4,
 6,
 0,
 4,
 6,
 0,
 4,
 6,
 0,
 4,
 6,
 0,
 4,
 6,
 0,
 4,
 6,
 0,
 4,
 6,
 0,
 4,
 6,
 0,
 4,
 6,
 0,
 4,
 6,
 0,
 4,
 6,
 0,
 4,
 6,
 0,
 4,
 6,
 0,
 4,
 6,
 0,
 4,
 6,
 0,
 4,
 6,
 0,
 4,
 6,
 0,
 4,
 6,
 0,
 4,
 6,
 0,
 4,
 6,
 0,
 4,
 6,
 0,
 4,
 6,
 0,
 4,
 6,
 0,
 4,
 6,
 0,
 4,
 6,
 0]

In [60]:
reward_[8]

[70,
 72,
 86,
 79.0,
 68.8,
 93.4,
 87.10000000000001,
 65.92,
 100.06,
 94.39000000000001,
 63.328,
 106.054,
 100.95100000000002,
 60.995200000000004,
 111.4486,
 106.85590000000002,
 58.895680000000006,
 116.30374,
 112.17031000000001,
 57.00611200000001,
 120.673366,
 116.95327900000001,
 55.30550080000001,
 124.6060294,
 121.25795110000001,
 53.774950720000014,
 128.14542646,
 125.13215599000002,
 52.39745564800001,
 131.330883814,
 128.61894039100002,
 51.15771008320001,
 134.1977954326,
 131.75704635190004,
 50.041939074880005,
 136.77801588934,
 134.58134171671003,
 49.037745167392,
 139.10021430040598,
 137.123207545039,
 48.133970650652806,
 141.19019287036537,
 139.41088679053513,
 47.320573585587525,
 143.07117358332883,
 141.46979811148162,
 46.58851622702878,
 144.76405622499595,
 143.32281830033347,
 45.9296646043259,
 146.28765060249637,
 144.99053647030013,
 45.33669814389331,
 147.65888554224674,
 146.49148282327013,
 44.80302832950398,
 148.89299698802208,
 147.8423